# Model Testing

In [34]:
import re
import pandas as pd
import numpy as np

## Data Import

In [55]:
file2=pd.read_csv('Data3.csv', encoding='ISO-8859-1', error_bad_lines=False)
pairs=np.column_stack((file2.loc[:998, ["SR Summary"]]["SR Summary"], file2.loc[:998, ["Group Issue"]]["Group Issue"]))
features_used = np.genfromtxt('features.csv', delimiter=",", dtype=None).astype(np.dtype(str))
pairs

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (25,43,46) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  This is separate from the ipykernel package so we can avoid doing imports until


array([['COMPLETE FOOT PEDAL WIRING', '1'],
       ['inspection, dielectric. leak at digger(fitting in cab) ,remove extension shaft from winch,emg. stop not working at controls, boom cuts out(wait or play with outriggers works)',
        '3,'],
       ['ANNUAL PM, DIELECTRIC AND DOT INSPECTION', '2,3,'],
       ...,
       ['INSTALL GPS IN CHASSIS', '11'],
       ['INTRA - INSTALL GPS IN TRUCK', '11'],
       ['INTRA - INSTALL GPS AND PERFORM DOT INSPECTION', '11']],
      dtype=object)

In [56]:
pairs[:,0]=[re.sub(r"  +"," ",re.sub(r"(?![\w\s]).", " ",line).lower()).strip() for line in pairs[:,0]]
triples = np.column_stack((pairs, [re.sub(r"  +"," ",re.sub(r"( (and|or|not) )((and|or|not) )*"," ",line)).strip() for line in pairs[:,0]]))
triples[1,0]

'inspection dielectric leak at digger fitting in cab remove extension shaft from winch emg stop not working at controls boom cuts out wait or play with outriggers works'

In [57]:
triples['2' == triples[:,1],1] = '23'
triples['3' == triples[:,1],1] = '23'
triples['2,3' == triples[:,1],1] = '23'

In [58]:
def containsComma(array):
    x = []
    i = 0
    for row in array:
        if type(row[1]) == str and not "," in row[1]: 
            x += [row]
        i +=1
    return x

data=containsComma(triples.tolist())
data=np.array([np.array(row) for row in data])
data

array([['complete foot pedal wiring', '1', 'complete foot pedal wiring'],
       ['service request', '5', 'service request'],
       ['service request', '5', 'service request'],
       ...,
       ['install gps in chassis', '11', 'install gps in chassis'],
       ['intra install gps in truck', '11', 'intra install gps in truck'],
       ['intra install gps and perform dot inspection', '11',
        'intra install gps perform dot inspection']], dtype='<U126')

In [59]:
binary=[]
for index in range(0, len(data)):
    binary += [[int(feature in data[index,0]) for feature in features_used[:-20]] + [int(feature in data[index,2]) for feature in features_used[-20:]]]
    
final_data = []
for index in range(0, len(data)):
    final_data += [[binary[index], data[index, 1]]]

binary[0]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

## Stochastic Gradient Descent

In [60]:
from sklearn.linear_model import SGDClassifier
SGD = SGDClassifier(random_state=0).fit(binary[:int(0.9 * len(binary))], data[:int(0.9 * len(binary)),1])
SGD.score(binary[int(0.9 * len(binary)):], data[int(0.9 * len(binary)):,1])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.19387755102040816

## Support Vector Classifier

In [61]:
from sklearn.svm import SVC
SVM = SVC(random_state=0).fit(binary[:int(0.9 * len(binary))], data[:int(0.9 * len(binary)),1])
SVM.score(binary[int(0.9 * len(binary)):], data[int(0.9 * len(binary)):,1])

0.1836734693877551

## Decision Tree Classifier

In [62]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier(random_state=0).fit(binary[:int(0.9 * len(binary))], data[:int(0.9 * len(binary)),1])
DT.score(binary[int(0.9 * len(binary)):], data[int(0.9 * len(binary)):,1])

0.15306122448979592

## Gradient Tree Boosting Classifier

In [63]:
from sklearn.ensemble import GradientBoostingClassifier
GBTC = GradientBoostingClassifier(random_state=0).fit(binary[:int(0.9 * len(binary))], data[:int(0.9 * len(binary)),1])
GBTC.score(binary[int(0.9 * len(binary)):], data[int(0.9 * len(binary)):,1])

0.17346938775510204

## Random Forest Classifier

In [64]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(random_state=0).fit(binary[:int(0.9 * len(binary))], data[:int(0.9 * len(binary)),1])
RFC.score(binary[int(0.9 * len(binary)):], data[int(0.9 * len(binary)):,1])

0.19387755102040816